# Basset
Old code home: https://github.com/davek44/Basset <br>
New code with many other improvements and other projects are in: https://github.com/calico/basenji

Paper: https://genome.cshlp.org/content/26/7/990.full.pdf+html (2016)<br>
follow up paper: https://genome.cshlp.org/content/28/5/739.full.pdf+html (2018) <br>

\[From the Github page\] <br>
Basset provides researchers with tools to: <br>
*   Train deep convolutional neural networks to learn highly accurate models of DNA sequence activity such as accessibility (via DNaseI-seq or ATAC-seq), protein binding (via ChIP-seq), and chromatin state.
*   Interpret the principles learned by the model.<br>

The input will be a BED (Browser Extensible Data) file. Store genomic regions as coordinates and associated annotations. This format was developed during the human Genome project. 

The file is a BED file containing data from NGS technique known as ATAC-seq. For more datails go to:<br>
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE47753 <br>
Design: We examined chromatin structure using ATAC-seq in 2 cell types (GM12878 cell line, purified CD4+ T cells).<br>
The below data is for the T cells only. 

# Main points of this notebook:
<b>Download</b> the dataset (only the T cell dataset) from ENCODE. Also the human reference dataset will be downloaded.<br>
<b>Then</b> from the Encode set we extract the negative examples as the TCell dataset only contains positive examples. This dataset also contains a activity file which gives pointer to which range corresponds to which cell (164 cell type in total) type. <br>
<b>Extract</b> negative examples from the file and also from the activity file.<br>
<b>Then</b> create a BED file containing all the exaples positive and negative and a corresponding activity file conatining a one hot encoded pointer to which range is used by which cell type. <br>
<b>The peaks</b> which overlapped greater than some value are merged down. <br>
<b>A bedtool</b> software is downloaded which helps in creating a fasta file which contains the base pairs (ATGC) for the corresponding range, which is around 600 base pair for each range. <br>


In [ ]:
# Get the CD4 Tcell
!wget ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE47nnn/GSE47753/suppl/GSE47753%5FCD4%2B%5FATACseq%5FAllDays%5FAllReps%5FZINBA%5Fpp08%2Ebed%2Egz

In [ ]:
# rename the file
!mv GSE47753_CD4+_ATACseq_AllDays_AllReps_ZINBA_pp08.bed.gz atac_cd4.bed.gz

In [ ]:
#unzip it and remove the compressed file.
!gunzip -f atac_cd4.bed.gz

In [ ]:
# Here we are using only one BED file, it may happen that there are multiple BED files
# required in a study than in this file cd4_sample.txt will have all the file names 
# written on it?
samples_out = open('cd4_sample.txt', 'w')
#print >> samples_out, 'CD4+\tatac_cd4.bed'
print('CD4+\tatac_cd4.bed',file=samples_out)
samples_out.close()

The content of the above file if for T cell only, so these are all positive inputs or training examples, we also need to feed negative training examples. Look at the data carefully and see whether it is diverse enough that the file itself contain negative training data, If not grab some negative data. In this case the negative training examples are taken from compendium of ENCODE and Epigenomic Roadmap data. The data is built around NGS to map DNA Methylation, histone modification, chromatin accessibility and others (http://www.roadmapepigenomics.org/). 

There is about 43K positive data points in the above BED file. Need to grab around 50K negative exaples. To do so lets grab the data that the author provide via the his dropbox, these files are from ENCODE epigenomic project (?). To do so run the file install_data.py (Code from original code base https://github.com/davek44/Basset)

This file does two things one is that it download the human reference genome, and also download the negative examples. Lets look at the human genome first. Run the code to download it. 

The Human Genome can be downloaded from <code>ftp://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/chromFa.tar.gz</code> it is around 1GB and after unzipping it become around 3GB. Here in this code the human reference genome hg19 is used, it was released in Feb 2009. Last version name is GRCh38 (Genome Reference Consortium) and UCSC version name hg38, released in 2013, this build have around 250 gaps, whereas the first version got 150000 gaps. 

The public database is downloaded file from the author's dropbox account. Two files to download:
*  https://www.dropbox.com/s/h1cqokbr8vjj5wc/encode_roadmap.bed.gz
*  https://www.dropbox.com/s/8g3kc0ai9ir5d15/encode_roadmap_act.txt.gz

See below: 

In [ ]:
!python3 /content/install_data.py

In [ ]:
ls -lh /content/data/genomes

total 3.0G
-rw-r--r-- 1 root root 3.0G May  9 08:59 hg19.fa
-rw-r--r-- 1 root root  788 May  9 09:00 hg19.fa.fai


The files are small like 30MB but encode_roadmap.bed file is about 200MB after unzipping and activation table file is around 600MB after uncompresion it. It is a BED file it represents:<br>
1st column--The Chromosome number<br>
2nd column-- Start coordinate of the chromosome (the first base is numbered 0)<br>
3rd column-- End coordinate of the chromosome<br>
4th column-- Name of the line in BED file<br>
5th column-- score, in between 0 and 1000<br>
6th column-- DNA strand orientation, + or -<br>
And final column contains infromation about: My assesment- there is another file called activity (act) file which contains info about what these comma seperated values represent. In the act file various fields are chromosome number, start, and end. In a single row there are around 150 different cell type information. These 150 position are one hot encoded for each cell type.  <br>
Both of these files are shown below. The act file is around 600MB uncompressed. In compressed state it is around 40MB.<br>

In [ ]:
!head encode_roadmap.bed

chr13	19144320	19144920	.	1	+	4
chr13	19168795	19169395	.	1	+	72
chr13	19170000	19170600	.	1	+	25
chr13	19172116	19172716	.	1	+	2,8,10,21,26,29,48,118
chr13	19172606	19173206	.	1	+	0,2,21,22,24,36,38,116,121
chr13	19173846	19174446	.	1	+	0,2,4,5,7,8,10,11,19,20,21,22,23,24,26,29,31,36,37,38,39,46,99,108,113,115,116,120,121,122,124
chr13	19174308	19174908	.	1	+	0,3,4,10,37,116
chr13	19183194	19183794	.	1	+	2,4,19,21,24,29,32,33,35,37,38,39,48,113,115,116,119,121,124
chr13	19184460	19185060	.	1	+	21
chr13	19185331	19185931	.	1	+	0,25,29,39,116


In [ ]:
!head encode_roadmap_act.txt

	8988T	AoSMC	Chorion	CLL	Fibrobl	FibroP	Gliobla	GM12891	GM12892	GM18507	GM19238	GM19239	GM19240	H9ES	HeLa-S3_IFNa4h	Hepatocytes	HPDE6-E6E7	HSMM_emb	HTR8svn	Huh-7.5	Huh-7	iPS	Ishikawa_Estradiol	Ishikawa_4OHTAM	LNCaP_androgen	MCF-7_Hypoxia	Medullo	Melano	Myometr	Osteobl	PanIsletD	PanIslets	pHTE	ProgFib	RWPE1	Stellate	T-47D	CD4_Th0	Urothelia	Urothelia_UT189	AG04449	AG04450	AG09309	AG09319	AG10803	AoAF	BE2_C	BJ	Caco-2	CD20+	CD34+	CMK	GM06990	GM12864	GM12865	H7-hESC	HAc	HAEpiC	HA-h	HA-sp	HBMEC	HCF	HCFaa	HCM	HConF	HCPEpiC	HCT-116	HEEpiC	HFF	HFF-Myc	HGF	HIPEpiC	HL-60	HMF	HMVEC-dAd	HMVEC-dBl-Ad	HMVEC-dBl-Neo	HMVEC-dLy-Ad	HMVEC-dLy-Neo	HMVEC-dNeo	HMVEC-LBl	HMVEC-LLy	HNPCEpiC	HPAEC	HPAF	HPdLF	HPF	HRCEpiC	HRE	HRGEC	HRPEpiC	HVMF	Jurkat	Monocytes-CD14+	NB4	NH-A	NHDF-Ad	NHDF-neo	NHLF	NT2-D1	PANC-1	PrEC	RPTEC	SAEC	SKMC	SK-N-MC	SK-N-SH_RA	Th2	WERI-Rb-1	WI-38	WI-38_4OHTAM	A549	GM12878	H1-hESC	HeLa-S3	HepG2	HMEC	HSMM	HSMMtube	HUVEC	K562	LNCaP	MCF-7	NHEK	Th1	LNG.IMR90 	ESC.H9 	ESC.H1 	IPSC.DF.6.9 	IPSC.D

In [ ]:
!apt-get install bedtools

As already mentioned the negative examples are taken from the ENCODE roadmap project, these is a total of almost 43K positive cases, we can take 50K negative cases also. After getting the negative cases preprocessing is done. This process create two files one called neg.bed and another called neg_act.txt which contains the activity text (cell type) of the BED file. 

In [ ]:
!python3 basset_sample.py /content/data/encode_roadmap.bed /content/data/encode_roadmap_act.txt 50000 neg

In [ ]:
!python2 preprocess_features.py -y -m 200 -s 600 -b neg.bed -n -o learn_cd4 -c /content/data/genomes/human.hg19.genome cd4_sample.txt

Ignoring chrY +


In [ ]:
!bedtools getfasta -fi /content/data/genomes/hg19.fa -bed learn_cd4.bed -s -fo learn_cd4.fa

In [ ]:
!python3 seq_hdf5.py -c -t 71886 -v 70000 encode_roadmap.fa encode_roadmap_act.txt encode_roadmap.h5

Ignoring header line
tcmalloc: large alloc 4852531200 bytes == 0x55e806958000 @  0x7fc7b2bb91e7 0x7fc7aee0c46e 0x7fc7aee5cc7b 0x7fc7aee5cd18 0x7fc7aef04010 0x7fc7aef0473c 0x7fc7aef0485d 0x55e5e8677f68 0x7fc7aee49ef7 0x55e5e8675c47 0x55e5e8675a50 0x55e5e86e9453 0x55e5e86e44ae 0x55e5e86773ea 0x55e5e86e97f0 0x55e5e85b6d14 0x7fc7aee49ef7 0x55e5e8675c47 0x55e5e8675a50 0x55e5e86e9453 0x55e5e86e44ae 0x55e5e86773ea 0x55e5e86e97f0 0x55e5e86e44ae 0x55e5e86773ea 0x55e5e86e53b5 0x55e5e86e44ae 0x55e5e86773ea 0x55e5e86e632a 0x55e5e867730a 0x55e5e86e53b5
^C


In [ ]:
mv /content/data/encode_roadmap.fa /content

In [ ]:
mv /content/data/encode_roadmap_act.txt /content

In [ ]:
!python3 /content/data/basset_sample.py /content/data/encode_roadmap.bed /content/data/encode_roadmap_act.txt 50000 neg

In [ ]:
mv /content/encode_roadmap_act.txt /content/data

In [ ]:
!head /content/neg.bed

chr12	25849135	25849735	.	1	+	94

chr20	54625795	54626395	.	1	+	162

chr9	87612900	87613500	.	1	+	6

chr11	118598746	118599346	.	1	+	0,51,55,66,113,128

chr1	224568285	224568885	.	1	+	92,141,161



In [ ]:
!wget https://www.dropbox.com/s/h1cqokbr8vjj5wc/encode_roadmap.bed.gz

In [ ]:
!gunzip -f encode_roadmap.bed.gz

In [ ]:
!wget  https://www.dropbox.com/s/8g3kc0ai9ir5d15/encode_roadmap_act.txt.gz

In [ ]:
!gunzip -f encode_roadmap_act.txt.gz

In [ ]:
!wc -l encode_roadmap_act.txt

2021887 encode_roadmap_act.txt


In [ ]:
!wc -l encode_roadmap.bed

2021886 encode_roadmap.bed


In [ ]:
!python3 basset_sample.py /content/encode_roadmap.bed /content/encode_roadmap_act.txt 40000 neg

In [ ]:
!tail atac_cd4.bed

chr21	47743501	47745000	id.42740	1000	+	101	15.95458977019100	16.00000000000000
chr21	47878276	47879325	id.42741	1000	+	73	15.65355977452702	16.00000000000000
chr21	47970301	47971050	id.42742	1000	+	48	8.17176408489583	9.29326336167170
chr21	47971576	47971950	id.42743	1000	+	21	0.87541278566652	1.64203877479696
chr21	47972251	47972700	id.42744	1000	+	28	2.59697826353454	3.57546150436463
chr21	47979226	47979825	id.42745	1000	+	36	4.74861218855765	5.94292471958139
chr21	48026026	48026325	id.42746	1000	+	21	0.87213592050239	1.66772359567759
chr21	48046576	48046875	id.42747	1000	+	21	0.87541278566652	1.63978310477478
chr21	48053476	48053850	id.42748	1000	+	23	1.32852495610945	2.18739293891445
chr21	48055201	48055875	id.42749	1000	+	52	9.26555534042431	10.45296656504838


In [ ]:
!tail neg_act.txt

chr17:81023417-81024017(+)	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0

chr17:81037184-81037784(+)	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	0	1	1	1	1	1	1	0	1	1	1	1	0	0	1	1	1	1	1	0	0	1	0	1	1	1	1	1	1	1	1	0	1	1	1	1	0	1	1	1	1	1	0	1	1	1	1	0	0	0	1	0	0	1	0	0	1	1	0	1	0	0	0	0	1	0	1	1	1	1	0	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	0	1	1	1	1	0	0	1	1	1	1	1	1	1	1	1	1	0	0	0	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	1

chr17:81075241-81075841(+)	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	1	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	1	0	1	0	0	0	0

# A Detailed review of the <code>basset_sample.py</code> code


<code>!python3 basset_sample.py /content/data/encode_roadmap.bed /content/data/encode_roadmap_act.txt 50000 neg</code><br>


In [ ]:
# Number of negative samples
Num_of_samples=50000

In [ ]:
# create a list of buffer. In this case it has only 50000 elements but for very large size will this approach be feasible?
buffer=['']*Num_of_samples

In [ ]:
len(buffer)

50000

In [ ]:
# The ENCODE roadmap BED file
bed_in=open('//content//encode_roadmap.bed')

In [ ]:
# Populate the buffer with first Num_of_samples.
i=0
while i< Num_of_samples:
  buffer[i]=bed_in.readline()
  i+=1

In [ ]:
import random
for line in bed_in:
  j=random.randint(0,i+1)
  if j<Num_of_samples:
    buffer[j]=line
  i+=1

bed_in.close()

In [ ]:
bed_out=open('neg.bed','w')
for r in range(len(buffer)):
  print(buffer[r][0:-1],file=bed_out)

bed_out.close()

In [ ]:
# Now have to select those rows from the act file that are already selected randomly
# in the corresponding BED file. So create a set and put the various parameters in it
# and scan the parameters against that in the act file (for cell type), as done in the last for loop.
buffer_header=set()
for line in buffer:
  a=line.rstrip().split('\t')
  chrom=a[0]
  start=a[1]
  end=a[2]
  strand=a[5]
  header='%s:%s-%s(%s)'%(chrom,start,end,strand)
  buffer_header.add(header)

act_out=open('neg_act.txt','w')
act_in=open('//content//encode_roadmap_act.txt')

print(act_in.readline(),file=act_out)

for line in act_in:
  a=line.split('\t')
  if a[0] in buffer_header:
    print(line[:-1],file=act_out)

act_in.close()
act_out.close()


In [ ]:
!cat neg_act1.txt

In [ ]:
!head neg.bed

In [ ]:
buffer_header

In [ ]:
!head learn_cd4.fa

>chr13:19572413-19573013(+)
GTCCCTCCTGGGCAGCTCTTCACACCAGCTCATAGGACCTGGAGCTCACCTGCACTCGGGATTGGGGCAGCCTCCTCCTTCCCTCCGCTTCTCCTCTCCTGGCTCTGAGCATTCACATCTGATTTGGAGAGCTGATAACCTGTTTGCAGAAAACACGTACCTGAATGTTTCGTTCTGTGCTGAGATCTGTGCCTGATCTTCAAGGGCAGGGAGGGGAGGAACACAGCACACATGAAAGGGCAATGCCAGGAAGAGCGCCCTAAATACCAGAGACATGGGAGGCCCCGCCCAGAGCAGGAAGGGCACCTGGTCTTGAGCTCAGAGACTGGAGGGTTGTTTGCTGATGCCCATGCCCAGGGCTAAAGACAGAATGCACggccaggcatggtggcccctgcctgtaactgcagcactttgggagattgtgacttgaggccaggagttcaaggcctgtctgggcaacatacagatcttgtctccaaaagaatagaaaattagttaggcgtagtggtgtatgtctgtagtcccagctactcaggaagctgaggcaggaggatcacctgagcccagaaatttgaggctgcagtgagctgattgtgc
>chr13:19630725-19631325(+)
TCAGAGACCACGTCAGGACTGAGGCCTCCTGCACCAGAACAAGTGTGGACAGGGCCTCAGTGCTGGAGAGGCGAGCCTGGGTCTGGCCACCCACCTCCACAGTCACAGGAGGAGGCCTGGCTGCGGTGCCGCATCTGGGACCATAGGGCAGCCAGGGCAGATGCAGTGCTGAATCCAGAACCCCACGTGTCAGATGGAGACGCTGAGCCTGTCCACCCCACGTACTTCCTCACACTGCCCAGGTCACACCTTCTCCTGGGGCACCAGGTACGGACATCTTAGACTTTTTACATGTGCATGTGAGCACATGTGCACACATCTCTAGTGGACTATGTGCCCATGC

In [ ]:
!head learn_cd4.bed

chr13	19572413	19573013	.	1	+	0
chr13	19630725	19631325	.	1	+	0
chr13	19645200	19645800	.	1	+	0
chr13	19726163	19726763	.	1	+	0
chr13	19729575	19730175	.	1	+	0
chr13	19730700	19731300	.	1	+	0
chr13	20161313	20161913	.	1	+	0
chr13	20207625	20208225	.	1	+	0
chr13	20356538	20357138	.	1	+	0
chr13	20391638	20392238	.	1	+	0


In [ ]:
!head learn_cd4_act.txt

	CD4+
chr13:19572413-19573013(+)	1
chr13:19630725-19631325(+)	1
chr13:19645200-19645800(+)	1
chr13:19726163-19726763(+)	1
chr13:19729575-19730175(+)	1
chr13:19730700-19731300(+)	1
chr13:20161313-20161913(+)	1
chr13:20207625-20208225(+)	1
chr13:20356538-20357138(+)	1


In [ ]:
!head /content/data/genomes/hg19.fa.fai

chr1	249250621	6	50	51
chr2	243199373	254235646	50	51
chr3	198022430	502299013	50	51
chr4	191154276	704281898	50	51
chr5	180915260	899259266	50	51
chr6	171115067	1083792838	50	51
chr7	159138663	1258330213	50	51
chr8	146364022	1420651656	50	51
chr9	141213431	1569942965	50	51
chrM	16571	1713980671	50	51


In [ ]:
!python2 preprocess_features.py -y -m 200 -s 600 -b neg.bed -n -o learn_cd4 -c /content/data/genomes/human.hg19.genome cd4_sample.txt

In [ ]:
!python2 preprocess_features.py -y -m 200 -s 600 -a /content/encode_roadmap_act.txt -b /content/encode_roadmap.bed -o db_cd4 -c /content/data/genomes/human.hg19.genome cd4_sample.txt

In [ ]:
# This list contains all the target cells? like here in this case CD4+, if more cases are included
# this will be populated with other types.
db_targets=[]

In [ ]:
# If MULTIPLE input with the whole activation file, for multitask learning. use below codes
db_add=False
db_bed=True
no_db_activity=False
db_act_file='/content/neg_act.txt'
if db_bed:
  db_add=True
  if not no_db_activity:
    if db_act_file is None:
      print("The activity file is not provided")
    else:
      db_act_in=open(db_act_file)
      db_targets=db_act_in.readline().strip().split('\t')
      db_act_in.close()


In [ ]:
# For isolated file 
db_add=False

In [ ]:
len(db_targets)

0

In [ ]:
# This contains the name of the beds file that will be include in this experimentation. It may contain all
# the positive examples bed files or if the file happen to have enough diversity then it may not need
# any negative example file. But in this example there is another file for the negative examples that will
# be appended to this list later on.
target_beds=[]

In [ ]:
# This is only a list of indexes for the files that we have till now.
target_dbi=[]

In [ ]:
# This code actually segregate the target types and the name of the bed files.

target_beds_file="cd4_sample.txt"
for line in open(target_beds_file):
  a=line.split()
  if len(a) !=2:
    print("each row of the target BED must contain a label")
  target_dbi.append(len(db_targets))
  db_targets.append(a[0])
  target_beds.append(a[1])

In [ ]:
# for this example the only file is the atac_cd4 bed file
target_beds

['atac_cd4.bed']

In [ ]:
# the target is the CD4 T-cell so it is the only one used in this example.
db_targets

['CD4+']

In [ ]:
#for multiple cells
target_dbi,len(db_targets)

([164], 165)

In [ ]:
target_dbi,len(db_targets)

([0], 1)

In [ ]:
chromosome_length_file="/content/data/genomes/human.hg19.genome"
chromosome_length={}
for line in open(chromosome_length_file):
  a=line.split()
  chromosome_length[a[0]]=int(a[1])

In [ ]:
ext_len=600 # need to be given as an input
peak_beds=target_beds

In [ ]:
peak_beds.append('neg.bed')

In [ ]:
peak_beds

['atac_cd4.bed', 'neg.bed']

In [ ]:
def mid_point(start,end):
  return int((start+end)/2)

In [ ]:
chrom_files={}
chrom_outs={}

In [ ]:
# Now open each of the bed file
for index in range(len(peak_beds)):
  peak_beds_in=open(peak_beds[index])

  for line in peak_beds_in:
    if not line.startswith('#'):
      a=line.split('\t')
      # Below line only remove the \n at the end of each row. 
      a[-1]=a[-1].rstrip()
      
      # a tupe of key is created using the chromosome number (e.g chr2) and the strand
      # the fifth colum is for the strand orientation and by default if no info is given
      # then by default + strand is considered.
      chrom=a[0]
      strand='+'
      if len(a)>5 and a[5] in '+-':
        strand=a[5]
      chrom_key=(chrom,strand)

      # Adjust coordinates to midpoint but why????? Also the value put into a[1]
      # and a[2] need to be integer.
      start=int(a[1])
      end=int(a[2])
      mid=mid_point(start,end)
      a[1]=str(mid)
      a[2]=str(mid+1)
      # for each chromosome and the strand a seperate file is created, chrom_files contains the 
      # info about the the chrosome type and the name of the file. chrom_out store the chromosome 
      # name, starnd and the actual file handle. 
      if chrom_key not in chrom_outs:
        chrom_files[chrom_key]='%s_%s_%s.bed' % ('learn_cd4',chrom,strand)
        chrom_outs[chrom_key]=open(chrom_files[chrom_key],'w')

      # if it is the database bed file with a activity file
      '''if db_add and index==len(peak_beds)-1:
        if no_db_activity:
          a[6]='.'
          print('\t'.join(a[:7]),file=chrom_outs[chrom_key])
        else:
          print(line.rstrip(),file=chrom_outs[chrom_key]) #USE rstrip() to remove the \n at the end'''
      if index==len(peak_beds)-1:  #Comment this line for multiple file uncomment above codes
        a[6]='.'#Comment this line for multiple file
        print('\t'.join(a[:7]),file=chrom_outs[chrom_key])#Comment this line for multiple file
      else:
        while len(a)<7:
          a.append('')      
        a[5]=strand
        a[6]=str(target_dbi[index])
        print('\t'.join(a[:7]),file=chrom_outs[chrom_key])

peak_beds_in.close()
for chrom_key in chrom_outs:
  chrom_outs[chrom_key].close()

# One can ignore chromosomeY and chromosome which are not of type chr(Digit) or chr(x/y)

In [ ]:
#Write code to ignore Y chromosome and other auxiliary chromosome

## A note about the chrosome files, produced from the previous codes

A row of the file (chr10) loos like<br>
*  chr10	133388.0	133389.0	id.25996	1000	+	0<br>

and toward the end the ENCORE file chr10 rows are added which look like <br>
*  chr10	127793510.0	127793511.0	.	1	+	.<br>


In [ ]:
# bedtools is used to short the content chromosome wise.
!apt-get install bedtools

In [ ]:
# New files are created with a different name, the previous files will get deleted
# the sortBed command is used which is in installed bedtools package. in the last line
# the new file handel is updated in the chrom_files. But not in chrom_outs

import subprocess
import os
for chrom_key in chrom_files:
  chrom,strand=chrom_key
  chrom_sort_bed='%s_%s_%s_sort.bed'%('learn_cd4',chrom,strand)
  sort_cmd='bedtools sort -i %s > %s'%(chrom_files[chrom_key],chrom_sort_bed)
  subprocess.call(sort_cmd,shell=True)
  os.remove(chrom_files[chrom_key])
  chrom_files[chrom_key]=chrom_sort_bed

In [ ]:
# Parse the chromosome specific files. The main aim is to find the peaks with some
# overlap. Also the peak class have an extend function it which it extend the range upto
# ext_len (here 600 is used). But if you look at the data all the ranges are already in the
# 600 range. Also the peaks are merged if two ranges overlap for 200 or more base pairs. 
merge_overlap=200
final_bed_out = open('%s.bed' % ('learn_cd4'), 'w')

for chrom_key in chrom_files:
  chrom,strand=chrom_key

  open_peaks=[]
  for line in open(chrom_files[chrom_key]):
    a=line.split('\t')
    a[-1]=a[-1].rstrip()
    
    peak_start=int(a[1])
    peak_end=int(a[2])
    peak_act=activity_set(a[6])
    peak=Peak(peak_start,peak_end,peak_act)
    peak.extend(ext_len,chromosome_length.get(chrom,None))

    if len(open_peaks)==0:
      open_end=peak.end
      open_peaks=[peak]
    else:
      if open_end-merge_overlap <= peak.start:
        mpeaks=merge_peaks(open_peaks,ext_len,merge_overlap,chromosome_length.get(chrom,None))
        
        for mpeak in mpeaks:
          print(mpeak.bed_string(chrom,strand),file=final_bed_out)

        open_end=peak.end
        open_peaks=[peak]
      else:
        open_peaks.append(peak)
        open_end=max(open_end,peak.end)
  if len(open_peaks)>0:
    if len(open_peaks)>1:
      print("mpeak0",open_peaks[0].start,open_peaks[0].end,open_peaks[1].start,open_peaks[1].end)
    mpeaks=merge_peaks(open_peaks,ext_len,merge_overlap,chromosome_length.get(chrom,None))
    for mpeak in mpeaks:
      print(mpeak.bed_string(chrom,strand),file=final_bed_out)
      print("mpeak",mpeak.start,mpeak.end)

final_bed_out.close()

mpeak2 1376183.0 1376783.0 1376250.0 1376850.0
mpeak2.1 1376183.0 1376783.0
mpeak2 2128967.0 2129567.0 2129325.0 2129925.0
mpeak2.1 2128967.0 2129567.0
mpeak2 2130525.0 2131125.0 2130595.0 2131195.0
mpeak2.1 2130525.0 2131125.0
mpeak2 2345550.0 2346150.0 2345623.0 2346223.0
mpeak2.1 2345550.0 2346150.0
mpeak2 2573905.0 2574505.0 2574075.0 2574675.0
mpeak2.1 2573905.0 2574505.0
mpeak2 3515550.0 3516150.0 3515724.0 3516324.0
mpeak2.1 3515550.0 3516150.0
mpeak2 4786950.0 4787550.0 4787232.0 4787832.0
mpeak2.1 4786950.0 4787550.0
mpeak2 9970181.0 9970781.0 9970350.0 9970950.0
mpeak2.1 9970181.0 9970781.0
mpeak2 11159663.0 11160263.0 11159671.0 11160271.0
mpeak2.1 11159663.0 11160263.0
mpeak2 12494134.0 12494734.0 12494175.0 12494775.0
mpeak2.1 12494134.0 12494734.0
mpeak2 15755850.0 15756450.0 15756145.0 15756745.0
mpeak2.1 15755850.0 15756450.0
mpeak2 15929454.0 15930054.0 15929850.0 15930450.0
mpeak2.1 15929454.0 15930054.0
mpeak2 16173740.0 16174340.0 16173975.0 16174575.0
mpeak2.1 1617

## Interpreting the above output for merging of peaks:
If suppose two (or more?) adjacent rows are there and the difference between the end of one peak to the start of another peak is less than 600 base pairs and the minimum overlap is more than 200 base pairs then these two peaks are merged as seen from the output above.

In [ ]:
for chrom_key in chrom_files:
  os.remove(chrom_files[chrom_key])

In [ ]:
# An accompanying act file (with cell types) for the BED file is also needed.

final_act_out=open('%s_act.txt' % ('learn_cd4'), 'w')

cols=['']+db_targets
print('\t'.join(cols),file=final_act_out)

for line in open('%s.bed'%('learn_cd4')):
  a=line.rstrip().split('\t')
  peak_id='%s:%s-%s(%s)'%(a[0],a[1],a[2],a[5])

  peak_act=[0]*len(db_targets)
  for act in a[6].split(','):
    if act!='.':
      peak_act[int(act)]=1

  cols=[peak_id]+peak_act
  print('\t'.join([str(c) for c in cols]),file=final_act_out)
final_act_out.close()

In [ ]:
def activity_set(comma_separated_activity):
  act_string=[act for act in comma_separated_activity.split(',')]

  #1
  if act_string[-1]=='':
    act_string=act_string[:-1]

  if act_string[0]=='.':
    act_set=set()
  else:
    act_set=set([int(act) for act in act_string]) #2

  return act_set

In [ ]:
# the chromosome length is the number of base bair present in that chromosome
# for example chr1 contain almost 250 million base pairs so if you look at the 
# chr1 data in the file it is around 250 million. 
# 1 so that the value did not become
# a negative value when computing the right or left side of the peak, so now the new start
# is ext_len/2 basepair on the side. the end is ext_len/2 added to the start that is either 
# to the left side of zero or midpoint (considering a real line as analogy)
# second function is just seating a new bed type string from the existing data, if the
# activities are already from a comma seperated field then they will be reintroduced and 
# will be sorted and create a new CSV. Finally a tab is inserted (why not new line?)
# Function MERGE merges two peaks. the middle of the merged peaked is found out by a weighted
# average and new start end and combined activation is created and intitialized in the end.
from numpy import average

class Peak:

  def __init__(self,start,end,act):
    self.start=start
    self.end=end
    self.act=act

  def extend(self,ext_len,chromosome_length):
    mid=mid_point(self.start,self.end)
    self.start=max(0,mid-ext_len/2) #1
    self.end=self.start+ext_len
    if chromosome_length and self.end > chromosome_length:
      self.end=chromosome_length
      self.start=self.end-ext_len

  def bed_string(self,chrom,starnd):
    if len(self.act)==0:
      act_str='.'
    else:
      act_str=','.join([str(act) for act in sorted(list(self.act))])
    col=(chrom,str(int(self.start)),str(int(self.end)),'.','1',strand,act_str)
    return '\t'.join(col)

  def merge(self,peak2,ext_len,chrom_len):
    peak_mids=[mid_point(self.start,self.end)]
    peak_mids.append(mid_point(peak2.start,peak2.end))

    peak_weights=[1+len(self.act)]
    peak_weights.append(1+len(peak2.act))

    #WILL CREATE A FLOATING POINT VALUE AFTER THE MERGE.
    merg_mid=int(0.5+average(peak_mids,weights=peak_weights))

    merge_start=max(0,merg_mid-ext_len/2)
    merge_end=merge_start+ext_len
    if chrom_len and merge_end>chrom_len:
      merge_end=chrom_len
      merge_start=merge_end-ext_len

    merge_act=self.act|peak2.act

    self.start=merge_start
    self.end=merge_end
    self.ext=merge_act

In [ ]:
# Repeatedly find the closest adjacent peaks and consider merging them together
# until there are no more peaks we want to merge. As in the individual file for 
# each chromosome, starting coordinates are sorted so any thing under the max_overlap 
# will get merged.  

def merge_peaks(peaks,ext_len,merge_overlap,chromosome_length):
  max_overlap=merge_overlap
  while len(peaks) > 1 and max_overlap >=merge_overlap:
    # Finding the largest overlap
    max_i=0
    print("mpeak2",peaks[0].start,peaks[0].end,peaks[1].start,peaks[1].end)
    max_overlap=peaks[0].end - peaks[1].start
    for i in range(1,len(peaks)-1):
      peaks_overlap=peaks[i].end-peaks[i+1].start
      if peaks_overlap > max_overlap:
        max_i=i
        max_overlap=peaks_overlap

    if max_overlap >= merge_overlap:
      # merge peaks
      peaks[max_i].merge(peaks[max_i],ext_len,chromosome_length)
      # remove merged peak
      peaks=peaks[:max_i+1]+peaks[max_i+2:]
    print("mpeak2.1",peaks[0].start,peaks[0].end)

  return peaks

# The bottom line for the two files created in the above process:
The fasta file contains the range and the nucleotides. The activity (act) file in this experiment contains the range and all other things and the last column contains a zero and one (one hot encoding) the ones are from the file that is known for the positive examples and set to 1 and the negative examples are set to 0. In case for other experiments there may be other columns in the activity file which will be one hot encoded (it more like binary encoded)  

In [ ]:
!wc -l learn_cd4.bed

91544 learn_cd4.bed


In [ ]:
# To convert the ranges given in the BED files into DNA sequence.
!wget ftp://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/chromFa.tar.gz

In [ ]:
!mv /content/chromFa.tar.gz /content/data/genomes

In [ ]:
!tar -xzvf /content/data/genomes/chromFa.tar.gz

In [ ]:
!cat chr?.fa chr??.fa > hg19.fa

In [ ]:
# remove all the other single chromosome files and others.
import glob
#os.remove('/content/data/genomes/chromFa.tar.gz')
for chrom_fa in glob.glob('chr*.fa'):
  os.remove(chrom_fa)

In [ ]:
!bedtools getfasta -fi /content/hg19.fa -bed learn_cd4.bed -s -fo learn_cd4.fa

In [ ]:
!cp /content/learn_cd4_91K.fa /content/drive/MyDrive/data
!cp /content/learn_cd4_act_91K.txt /content/drive/MyDrive/data

In [ ]:
!wc -l hg19.fa

61913917 hg19.fa


In [ ]:
!head -11147 hg19.fa | tail +11147

TCCCCACTGTCGTGGATTTGTCGGCTCATCTTTAGTGCGCTCCCTGAAGA


In [ ]:
!sed -n '1000p' hg19.fa

TTCTCACTGCTCTGAGCATGAATTCAATATTTCAGGGCAAACTAACTGAA


In [ ]:
# around 50 basepairs in each row
a='TTCTCACTGCTCTGAGCATGAATTCAATATTTCAGGGCAAACTAACTGAA'
len(a)

50

In [ ]:
# total human Genome 3 billion base pairs
61913917*50

3095695850

In [ ]:
!cp /content/learn_cd4_chr21_+_sort.bed  /content/data